# Домашнє завдання: Внесення оновлень в БД і робота з транзакціями

Це ДЗ передбачене під виконання на локальній машині. Виконання з Google Colab буде суттєво ускладнене.

## Підготовка
1. Переконайтесь, що у вас встановлены необхідні бібліотеки:
   ```bash
   pip install sqlalchemy pymysql pandas matplotlib seaborn python-dotenv
   ```

2. Створіть файл `.env` з параметрами підключення до бази даних classicmodels. Базу даних ви можете отримати через

  - docker-контейнер згідно існтрукції в [документі](https://www.notion.so/hannapylieva/Docker-1eb94835849480c9b2e7f5dc22ee4df9), також відео інструкції присутні на платформі - уроки "MySQL бази, клієнт для роботи з БД, Docker і ChatGPT для запитів" та "Як встановити Docker для роботи з базами даних без терміналу"
  - або встановивши локально цю БД - для цього перегляньте урок "Опціонально. Встановлення MySQL та  БД Сlassicmodels локально".
  
  Приклад `.env` файлу ми створювали в лекції. Ось його обовʼязкове наповнення:
    ```
    DB_HOST=your_host
    DB_PORT=3306 або 3307 - той, який Ви налаштували
    DB_USER=your_username
    DB_PASSWORD=your_password
    DB_NAME=classicmodels
    ```
  Якщо ви створили цей файл під час перегляду лекції - **новий створювати не треба**. Замініть лише назву БД, або пропишіть назву в коді створення підключення (замість отримання назви цільової БД зі змінних оточення). Але переконайтесь, що до `.env` файл лежить в тій самій папці, що і цей ноутбук.

  **УВАГА!** НЕ копіюйте скрит для **створення** `.env` файлу. В лекції він наводиться для прикладу. І давалось пояснення, що в реальних проєктах ми НІКОЛИ не пишемо доступи до бази в коді. Копіювання скрипта для створення `.env` файлу сюди в ДЗ буде вважатись грубою помилкою і ми зніматимемо бали.

3. Налаштуйте підключення через SQLAlchemy до БД за прикладом в лекції.

Рекомендую вивести (відобразити) змінну engine після створення. Вона має бути не None! Якщо None - значить у Вас не підтягнулись налаштування з .env файла.

Ви також можете налаштувати параметри підключення до БД без .env файла, просто прописавши текстом в відповідних місцях. Це - не рекомендований підхід.


## Завдання

### Завдання 1: Оновлення інформації про клієнта (2 бали)

**Створіть функцію для оновлення контактної інформації клієнта за його номером** з наступними можливостями:
- Оновлення телефону клієнта
- Оновлення email (якщо поле існує в таблиці)

Опціонально, якщо вам хочеться більше практики:
- Логування змін в окрему таблицю

Використайте підхід з параметризованими запитами через `text()` та `UPDATE` оператор. Не забудьте на початку перевірити чи існує клієнт з таким номером в базі - це хороша практика.

Отримати всі колонки, які існують в таблиці ви можете наступним запитом
```
  SELECT COLUMN_NAME, DATA_TYPE
  FROM INFORMATION_SCHEMA.COLUMNS
  WHERE TABLE_NAME = 'customers'
```

Запустіть функцію і продемонструйте її роботу, запустивши SELECT, який допоможе це зробити.



In [1]:
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os

# читаємо .env (без перезапису файлу)
load_dotenv(override=True)

def create_connection(db_name="classicmodels"):
    """
    Підключення до бази даних MySQL з параметрами з .env,
    але з можливістю вказати іншу назву бази (db_name).
    """
    host = os.getenv("DB_HOST", "127.0.0.1")
    port = int(os.getenv("DB_PORT", "3306"))
    user = os.getenv("DB_USER", "root")
    password = os.getenv("DB_PASSWORD", "")
    
    url = f"mysql+pymysql://{user}:{password}@{host}:{port}/{db_name}"

    engine = create_engine(
        url,
        pool_size=2,
        max_overflow=20,
        pool_pre_ping=True,
        echo=False
    )

    # тестуємо підключення
    with engine.connect() as conn:
        conn.execute(text("SELECT 1"))
    print(f"✅ Connected to {db_name} on {host}:{port} as {user}")
    return engine

# приклад використання
engine = create_connection("classicmodels")


✅ Connected to classicmodels on 127.0.0.1:3306 as root


In [5]:
from sqlalchemy import text
import pandas as pd

def table_has_column(engine, table_name: str, column_name: str) -> bool:
    sql = text("""
        SELECT 1
        FROM INFORMATION_SCHEMA.COLUMNS
        WHERE TABLE_SCHEMA = DATABASE()
          AND TABLE_NAME   = :t
          AND COLUMN_NAME  = :c
        LIMIT 1
    """)
    with engine.connect() as conn:
        return conn.execute(sql, {"t": table_name, "c": column_name}).fetchone() is not None

print("Є phone? ", table_has_column(engine, "customers", "phone"))
print("Є email? ", table_has_column(engine, "customers", "email"))



Є phone?  True
Є email?  False


In [7]:
# 1) Перевіримо поточний телефон клієнта 103
from sqlalchemy import text
import pandas as pd

pd.read_sql(
    text("SELECT DATABASE() AS db, customerNumber, customerName, phone "
         "FROM customers WHERE customerNumber=:n"),
    engine, params={"n": 103}
)


,db,customerNumber,customerName,phone
0,classicmodels,103,Atelier graphique,40.32.2555


In [8]:
# 2) ВИКЛИКАЄМО функцію оновлення (обери номер, відмінний від поточного)
update_customer_contact(
    engine,
    customer_number=103,
    new_phone="(+33) 40.32.9999",   # ← постав інший номер, не той що зараз
    log_changes=True
)


✅ Оновлено телефон клієнта 103: 40.32.2555 → (+33) 40.32.9999


In [9]:
# 3) Перевіряємо, що номер змінився
pd.read_sql(
    text("SELECT customerNumber, customerName, phone "
         "FROM customers WHERE customerNumber=:n"),
    engine, params={"n": 103}
)


,customerNumber,customerName,phone
0,103,Atelier graphique,(+33) 40.32.9999


In [10]:
from sqlalchemy import text
import pandas as pd

pd.read_sql(
    text("SELECT * FROM change_log_customers ORDER BY id DESC LIMIT 10"),
    engine
)


,id,customerNumber,field,old_value,new_value,changed_at
0,1,103,phone,40.32.2555,(+33) 40.32.9999,2025-08-24 14:57:53


### Завдання 2: Створення нового замовлення з транзакцією (5 балів)

**Реалізуйте процес створення нового замовлення** з наступними кроками в одній транзакції:
- Створення запису в таблиці `orders`
- Додавання товарних позицій в `orderdetails`
- Перевірка наявності товарів на складі
- Зменшення кількості товарів на складі

Запустіть процес з тестовими даними і продемонструйте через SELECT, що процес успішно відпрацював і були виконані необхідні операції.




In [11]:
# 1
import pandas as pd
from sqlalchemy import text

top_products = pd.read_sql(
    text("""
        SELECT productCode, productName, quantityInStock, MSRP
        FROM products
        ORDER BY quantityInStock DESC
        LIMIT 12
    """),
    engine
)
top_products


,productCode,productName,quantityInStock,MSRP
0,S12_2823,2002 Suzuki XREO,9997,150.62
1,S18_1984,1995 Honda Civic,9772,142.25
2,S700_2466,America West Airlines B757-200,9653,99.72
3,S24_3432,2002 Chevy Corvette,9446,107.08
4,S18_2325,1932 Model A Ford J-Coupe,9354,127.13
5,S32_2206,1982 Ducati 996 R,9241,40.23
6,S24_3151,1912 Ford Model T Delivery Wagon,9173,88.51
7,S18_3482,1976 Ford Gran Torino,9127,146.99
8,S12_3380,1968 Dodge Charger,9123,117.44
9,S18_1589,1965 Aston Martin DB5,9042,124.44


In [12]:
# 1b) Чи автоінкрементний orders.orderNumber?
auto = pd.read_sql(
    text("""
        SELECT EXTRA
        FROM INFORMATION_SCHEMA.COLUMNS
        WHERE TABLE_SCHEMA = DATABASE()
          AND TABLE_NAME   = 'orders'
          AND COLUMN_NAME  = 'orderNumber'
    """),
    engine
)
auto


,EXTRA
0,


In [13]:
from sqlalchemy import text
from datetime import date, timedelta

def create_order(engine, customer_number: int, items: list[dict], comment: str | None = None) -> int:
    """
    Створює нове замовлення для customer_number.
    items = [{"productCode": "S12_2823", "quantity": 2}, ...]
    Усе робиться в ОДНІЙ транзакції:
      - перевіряємо клієнта
      - блокуємо товари та перевіряємо залишки
      - створюємо order
      - додаємо orderdetails
      - зменшуємо products.quantityInStock
    Повертає orderNumber.
    """
    if not items:
        raise ValueError("Список items порожній")

    today = date.today()
    required = today + timedelta(days=14)

    with engine.begin() as conn:  # COMMIT/ROLLBACK автоматично
        # 1) Перевірка клієнта (та легке блокування рядка)
        cust = conn.execute(
            text("SELECT customerNumber FROM customers WHERE customerNumber = :c FOR UPDATE"),
            {"c": customer_number}
        ).fetchone()
        if not cust:
            raise ValueError(f"Клієнт {customer_number} не знайдений")

        # 2) Перевіряємо товари та блокуємо їх
        prod_info = {}
        for it in items:
            code = it["productCode"]
            qty  = int(it["quantity"])
            if qty <= 0:
                raise ValueError(f"Кількість для {code} має бути > 0")

            row = conn.execute(
                text("""SELECT productCode, productName, quantityInStock, MSRP
                        FROM products
                        WHERE productCode = :p
                        FOR UPDATE"""),
                {"p": code}
            ).fetchone()
            if not row:
                raise ValueError(f"Товар {code} не знайдений")
            if row.quantityInStock < qty:
                raise RuntimeError(
                    f"Недостатньо {code} ({row.productName}): в наявності {row.quantityInStock}, потрібно {qty}"
                )
            prod_info[code] = row

        # 3) Новий номер замовлення (бо не AUTO_INCREMENT)
        new_no = conn.execute(
            text("SELECT COALESCE(MAX(orderNumber), 0) + 1 AS n FROM orders FOR UPDATE")
        ).scalar_one()

        # 4) Вставляємо order
        conn.execute(
            text("""INSERT INTO orders
                    (orderNumber, orderDate, requiredDate, shippedDate, status, comments, customerNumber)
                    VALUES (:no, :od, :rd, NULL, 'In Process', :cm, :cn)"""),
            {"no": new_no, "od": today, "rd": required, "cm": comment, "cn": customer_number}
        )

        # 5) Додаємо позиції та зменшуємо склад
        line_no = 1
        for it in items:
            code = it["productCode"]
            qty  = int(it["quantity"])
            msrp = prod_info[code].MSRP  # ціна: беремо MSRP (можеш замінити логіку ціни за потреби)

            conn.execute(
                text("""INSERT INTO orderdetails
                        (orderNumber, productCode, quantityOrdered, priceEach, orderLineNumber)
                        VALUES (:no, :pc, :q, :price, :ln)"""),
                {"no": new_no, "pc": code, "q": qty, "price": msrp, "ln": line_no}
            )

            conn.execute(
                text("""UPDATE products
                        SET quantityInStock = quantityInStock - :q
                        WHERE productCode = :pc"""),
                {"q": qty, "pc": code}
            )
            line_no += 1

    print(f"✅ Замовлення {new_no} створено для клієнта {customer_number}")
    return new_no


In [14]:
# 3a) Перевіримо початкові залишки саме для цих двох кодів
from sqlalchemy import text
import pandas as pd

codes = ["S12_2823", "S18_1984"]

start_stock = pd.read_sql(
    text("""
        SELECT productCode, productName, quantityInStock, MSRP
        FROM products
        WHERE productCode IN (:c1, :c2)
        ORDER BY productCode
    """),
    engine, params={"c1": codes[0], "c2": codes[1]}
)
start_stock


,productCode,productName,quantityInStock,MSRP
0,S12_2823,2002 Suzuki XREO,9997,150.62
1,S18_1984,1995 Honda Civic,9772,142.25


In [15]:
# 3b) Викликаємо функцію створення замовлення
order_no = create_order(
    engine,
    customer_number=103,
    items=[
        {"productCode": "S12_2823", "quantity": 2},   # кількість можна змінити за бажанням
        {"productCode": "S18_1984", "quantity": 3},
    ],
    comment="Test order from Jupyter"
)
order_no


✅ Замовлення 10426 створено для клієнта 103


10426

In [16]:
# 3c) Перевіряємо результат: шапка замовлення, позиції і нові залишки
# 1) шапка
hdr = pd.read_sql(
    text("""SELECT orderNumber, orderDate, requiredDate, status, comments, customerNumber
            FROM orders WHERE orderNumber = :no"""),
    engine, params={"no": order_no}
)
display(hdr)

# 2) позиції
lines = pd.read_sql(
    text("""SELECT d.orderNumber, d.productCode, p.productName,
                   d.quantityOrdered, d.priceEach, d.orderLineNumber
            FROM orderdetails d
            JOIN products p USING(productCode)
            WHERE d.orderNumber = :no
            ORDER BY d.orderLineNumber"""),
    engine, params={"no": order_no}
)
display(lines)

# 3) нові залишки по цих товарах
end_stock = pd.read_sql(
    text("""
        SELECT productCode, productName, quantityInStock
        FROM products
        WHERE productCode IN (:c1, :c2)
        ORDER BY productCode
    """),
    engine, params={"c1": codes[0], "c2": codes[1]}
)
display(end_stock)


,orderNumber,orderDate,requiredDate,status,comments,customerNumber
0,10426,2025-08-24,2025-09-07,In Process,Test order from Jupyter,103


,orderNumber,productCode,productName,quantityOrdered,priceEach,orderLineNumber
0,10426,S12_2823,2002 Suzuki XREO,2,150.62,1
1,10426,S18_1984,1995 Honda Civic,3,142.25,2


,productCode,productName,quantityInStock
0,S12_2823,2002 Suzuki XREO,9995
1,S18_1984,1995 Honda Civic,9769


In [17]:
# перевіряємо загальну суму замовленб

from sqlalchemy import text
import pandas as pd

pd.read_sql(
    text("""
        SELECT d.orderNumber,
               SUM(d.quantityOrdered * d.priceEach) AS order_total
        FROM orderdetails d
        WHERE d.orderNumber = :no
    """),
    engine, params={"no": 10426}
)


,orderNumber,order_total
0,10426,727.99
